<a href="https://colab.research.google.com/github/sath08/researchProject/blob/main/researchBiasCorrection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cftime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.3 MB/s eta 0:00:00


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cython
import cftime
from scipy.io import netcdf
from tqdm import tqdm

In [ ]:
def fix_calendar(dataset):

  # Place all of the dates that should be removed into a boolean.
  removed_times = ~(((dataset.time.dt.month == 2) & (dataset.time.dt.day == 29)) | ((dataset.time.dt.month == 2) & (dataset.time.dt.day == 30)) | ((dataset.time.dt.month == 1) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 3) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 3) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 5) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 7) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 8) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 10) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 12) & (dataset.time.dt.day == 31)))

  # Return the dataset with the dates from removed_times excluded.
  return dataset.isel(time = removed_times)

In [ ]:
def perform_bias_correction ():
  # Open and calibrate past_gcm dataset.
  past_gcm = xr.open_dataset("/content/drive/MyDrive/Sathkrith_Segu_LumiereSummerProject/past_gcm/HADGEM_GCM_past.nc",decode_times=False)
  past_gcm = xr.decode_cf(past_gcm)
  past_gcm = fix_calendar(past_gcm)

  # Open and calibrate future_gcm dataset.
  future_gcm = xr.open_dataset("/content/drive/MyDrive/Sathkrith_Segu_LumiereSummerProject/future_gcm/ssp585/HADGEM_GCM_future.nc", decode_times=False)
  future_gcm = xr.decode_cf(future_gcm)
  future_gcm = fix_calendar(future_gcm)

  # Open era5 dataset.
  era5 = xr.open_dataset("/content/drive/MyDrive/Sathkrith_Segu_LumiereSummerProject/era5/ERA5_1980.nc")
  for i in range(34):
    era5CurrYear = xr.open_dataset("/content/drive/MyDrive/Sathkrith_Segu_LumiereSummerProject/era5/ERA5_" + str(1981 + i) + "_xarray.nc")
    era5 = xr.concat([era5, era5CurrYear], dim='time')

  # Adjust past_gcm and future_gcm datasets to match era5.
  past_gcm_2015_val = future_gcm.sel(time=slice('2015-01-01', '2015-01-01'))
  past_gcm = past_gcm.sel(time=slice('1980-01-01', '2014-12-30'))
  future_gcm = xr.concat([past_gcm_2015_val, future_gcm], dim='time')

  # 1:1 copy, not a reference, of the future_gcm so it can be modified.
  future_gcm_BC_copy = future_gcm.copy()

  # Dictionary to store variables for each month.
  month_variables = {}

  for i in tqdm(range(1,13)):

    # Variable name for each month.
    variable_name = f"month_{i}"

    # A month of data from the future to be bias corrected.
    future_gcm_BC_copy_month = future_gcm_BC_copy.sel(time=future_gcm_BC_copy['time.month'] == i)

    past_gcm_month = past_gcm.sel(time=past_gcm['time.month'] == i)['tas']
    future_gcm_month = future_gcm.sel(time=future_gcm['time.month'] == i)['tas']
    era5_gcm_month = era5.sel(time=era5['time.month'] == i)['t2m']

    # latitude loops
    for lat in range(len(past_gcm.lat)):

        # Converts the latitude value at index phi into a float in the variable lat_phi
        lat = past_gcm.lat[lat].values





# Bias corrected dataset setup
  bias_corrected_filepath = '/content/drive/MyDrive/Sathkrith_Segu_LumiereSummerProject/BC_dataset/HADGEM3_BC.nc'

  # Save the dataset to the updated file.
  # futr_PV_corr.to_netcdf(bias_corrected_filepath)


In [ ]:
perform_bias_correction()